In [1]:
from sklearn.neural_network import MLPRegressor
import mlflow
import os

In [5]:
# Establecer la URI de la base de datos SQLite y configuración de MLflow
os.environ['MLFLOW_TRACKING_URI'] = 'sqlite:///mlruns.db'
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("redes_neuronales")

<Experiment: artifact_location='mlflow-artifacts:/2', creation_time=1713175269356, experiment_id='2', last_update_time=1713175269356, lifecycle_stage='active', name='redes_neuronales', tags={'mlflow.sharedViewState.35fcf56d61b8dbb98512f6829913993e41257533dd332e1e8ff07999f73a21ea': '{"searchFilter":"","orderByKey":"attributes.start_time","orderByAsc":false,"startTime":"ALL","lifecycleFilter":"Active","datasetsFilter":[],"modelVersionFilter":"All '
                                                                                            'Runs","selectedColumns":["attributes.`Source`","attributes.`Models`","attributes.`Dataset`"],"runsExpanded":{},"runsPinned":[],"runsHidden":[],"runsHiddenMode":"FIRST_10_RUNS","compareRunCharts":[{"uuid":"1713177259273xowwzda7","type":"BAR","runsCountToCompare":10,"metricSectionId":"171317725927396vp3vfh","deleted":false,"isGenerated":true,"metricKey":"best_mean_test_accuracy"},{"uuid":"1713177259273vnosmd9g","type":"BAR","runsCountToCompare":10,"metri

# Sin Escalado | Con Hiperparámetros

In [7]:
with mlflow.start_run(run_name="evaluación_modelo_sin_escalado_hiperparametros"):
    import preprocesamiento_datos
    from sklearn.metrics import mean_absolute_error
    
    # Obtener los datos preprocesados
    X_train, X_val, X_test, y_train, y_val, y_test, RANDOM_STATE = preprocesamiento_datos.preprocesamiento(True,[], False)

    # Entrenamiento del modelo
    clf = MLPRegressor(random_state=RANDOM_STATE, solver="adam", alpha=0.01, learning_rate_init=0.001, hidden_layer_sizes=(24,))
    
    # Ajustar el modelo con los datos de entrenamiento
    clf.fit(X_train, y_train)
    
    # Predicción en el conjunto de validación
    val_pred = clf.predict(X_val)
    
    # Predicción en el conjunto de prueba
    test_pred = clf.predict(X_test)
    
    # Calcular métricas de evaluación en el conjunto de validación
    mae_val = mean_absolute_error(y_val, val_pred)
    
    # Calcular métricas de evaluación en el conjunto de prueba
    mae_test = mean_absolute_error(y_test, test_pred)
    
    print("Validation MAE:", mae_val)
    print("Test MAE:", mae_test)
    
    # Registro de hiperparámetros y métricas en MLflow
    mlflow.log_param("solver", 'adam')
    mlflow.log_param("alpha", 0.01)
    mlflow.log_param("batch_size", 16)
    mlflow.log_param("learning_rate_init", 0.1)
    mlflow.log_param("hidden_layer_sizes", (185,))
    mlflow.log_metric("val_mae", mae_val)
    mlflow.log_metric("test_mae", mae_test)

Archivo Datos_la_liga_preparados_entrenamiento.parquet guardado en: ../Downloads\Datos_la_liga_preparados_entrenamiento.parquet
Validation MAE: 2944768.087089949
Test MAE: 2714148.2625509985


# Hiperparámetros Escalado

In [13]:
with mlflow.start_run(run_name="MODELO FINAL"):
    import preprocesamiento_datos
    from sklearn.metrics import mean_absolute_error, max_error, r2_score
    
    # Obtener los datos preprocesados
    X_train, X_val, X_test, y_train, y_val, y_test, RANDOM_STATE = preprocesamiento_datos.preprocesamiento(False,[], False)
    
    # Entrenamiento del modelo
    clf = MLPRegressor(random_state=RANDOM_STATE, solver="adam", alpha=0.01, learning_rate_init=0.1, 
    batch_size= 16,
    hidden_layer_sizes=(185,))
    
    # Ajustar el modelo con los datos de entrenamiento
    clf.fit(X_train, y_train)
    
    # Predicción en el conjunto de validación
    val_pred = clf.predict(X_val)
    
    # Predicción en el conjunto de prueba
    test_pred = clf.predict(X_test)
    
    # Calcular métricas de evaluación en el conjunto de validación
    mae_val = mean_absolute_error(y_val, val_pred)
    
    # Calcular métricas de evaluación en el conjunto de prueba
    mae_test = mean_absolute_error(y_test, test_pred)
    
    print("Validation MAE:", mae_val)
    print("Test MAE:", mae_test)
    
    # Registro de hiperparámetros y métricas en MLflow
    mlflow.log_param("solver", 'adam')
    mlflow.log_param("alpha", 0.01)
    mlflow.log_param("batch_size", 16)
    mlflow.log_param("learning_rate_init", 0.1)
    mlflow.log_param("hidden_layer_sizes", (185,))
    mlflow.log_metric("val_mae", mae_val)
    mlflow.log_metric("test_mae", mae_test)

Archivo Datos_la_liga_preparados_entrenamiento.parquet guardado en: ../Downloads\Datos_la_liga_preparados_entrenamiento.parquet
Validation MAE: 3004416.477725017
Test MAE: 2629609.357214961


# Escalado | Default

In [12]:
with mlflow.start_run(run_name="evaluación_modelo_escalado_default"):
    import preprocesamiento_datos
    from sklearn.metrics import mean_absolute_error, max_error, r2_score
    
    # Obtener los datos preprocesados
    X_train, X_val, X_test, y_train, y_val, y_test, RANDOM_STATE = preprocesamiento_datos.preprocesamiento(False,[], False)
    
    # Entrenamiento del modelo
    clf = MLPRegressor(random_state=RANDOM_STATE)
    
    # Ajustar el modelo con los datos de entrenamiento
    clf.fit(X_train, y_train)
    
    # Predicción en el conjunto de validación
    val_pred = clf.predict(X_val)
    
    # Predicción en el conjunto de prueba
    test_pred = clf.predict(X_test)
    
    # Calcular métricas de evaluación en el conjunto de validación
    mae_val = mean_absolute_error(y_val, val_pred)
    
    # Calcular métricas de evaluación en el conjunto de prueba
    mae_test = mean_absolute_error(y_test, test_pred)
    
    print("Validation MAE:", mae_val)
    print("Test MAE:", mae_test)
    
    mlflow.log_metric("val_mae", mae_val)
    mlflow.log_metric("test_mae", mae_test)

Archivo Datos_la_liga_preparados_entrenamiento.parquet guardado en: ../Downloads\Datos_la_liga_preparados_entrenamiento.parquet
Validation MAE: 2932417.340993314
Test MAE: 2830651.9800939057


C:\Users\Usuario\Documents\Alvaro\Carrera\2-Segundo\Primer cuatrimetre\Fundamentos de la Inteligencia Artificial\PycharmProjects\proyecto\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
